In [ ]:
from monai.networks.nets import SPADEAutoencoderKL

/home/samirrajab1/.conda/envs/monai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from monai.networks.nets import PatchDiscriminator

In [ ]:
from dataset.get_data_df import BREASTSpadeDatasetDF
# df_dir = r"/projects/0/prjs1204/projects/spadebreast2025/src/spadebreast/dataset/combined_df/stratified_96_train_valid_df_seg.csv"
df_train='/projects/0/prjs1204/projects/spadebreast2025/src/spadebreast/dataset/combined_df/stratified_96_train_df_seg.csv'
df_valid ='/projects/0/prjs1204/projects/spadebreast2025/src/spadebreast/dataset/combined_df/stratified_96_valid_df_seg.csv'

train_ds = BREASTSpadeDatasetDF(df=df_train, phase='train')
validation_ds = BREASTSpadeDatasetDF(df=df_valid, phase='valid')


In [1]:
from dataset.get_data_df import BREASTSpadeDatasetDF_generate
df_dir = r"/projects/0/prjs1204/projects/spadebreast2025/src/spadebreast/dataset/combined_df/stratified_96_train_valid_df_seg.csv"
train_ds = BREASTSpadeDatasetDF_generate(df=df_dir, phase='train')
# validation_ds = BREASTSpadeDatasetDF(df=df_dir, phase='valid')

/home/samirrajab1/.conda/envs/monai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_ds, shuffle=False, batch_size=1,
                                num_workers=4)
    
print("size of train dataset: ", len(train_ds))

size of train dataset:  948


In [3]:
batch = next(iter(train_loader))
# print(batch['image'].shape)
# print(batch['label'].shape)


/home/samirrajab1/.conda/envs/monai/lib/python3.9/site-packages/torchio/data/image.py:251: UserWarning: Using TorchIO images without a torchio.SubjectsLoader in PyTorch >= 2.3 might have unexpected consequences, e.g., the collated batches will be instances of torchio.Subject with 5D images. Replace your PyTorch DataLoader with a torchio.SubjectsLoader so that the collated batch becomes a dictionary, as expected. See https://github.com/fepegar/torchio/issues/1179 for more context about this issue.
  warnings.warn(message, stacklevel=1)
/home/samirrajab1/.conda/envs/monai/lib/python3.9/site-packages/torchio/data/image.py:251: UserWarning: Using TorchIO images without a torchio.SubjectsLoader in PyTorch >= 2.3 might have unexpected consequences, e.g., the collated batches will be instances of torchio.Subject with 5D images. Replace your PyTorch DataLoader with a torchio.SubjectsLoader so that the collated batch becomes a dictionary, as expected. See https://github.com/fepegar/torchio/issu

In [9]:
batch["tio_sub"]['label'].data.shape

torch.Size([1, 1, 96, 96, 80])

In [10]:
import torch
images = torch.cat(
    [batch["tio_sub"][key].data for key in batch["tio_sub"].keys() if "image" in key],
    dim=0
)
print(images.shape)
# print(batch["tio_sub"]['label'].data.shape)

torch.Size([5, 1, 96, 96, 80])


In [11]:
from pathlib import Path
import os
image_file = Path(batch['images_path'][0])
image_dir = image_file.parent
images_path = sorted(os.listdir(image_dir))
images_path = [os.path.join(image_dir, image) for image in images_path]
print(images_path)

['/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/images/DUKE_001/duke_001_0000.nii.gz', '/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/images/DUKE_001/duke_001_0001.nii.gz', '/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/images/DUKE_001/duke_001_0002.nii.gz', '/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/images/DUKE_001/duke_001_0003.nii.gz', '/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/images/DUKE_001/duke_001_0004.nii.gz']


In [11]:
batch['data']

{'patient_id': ['DUKE_001'],
 'image_id': ['DUKE_001_0000'],
 'image_path': ['/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/images/DUKE_001/duke_001_0000.nii.gz'],
 'pcr': tensor([0.], dtype=torch.float64),
 'age': tensor([41.], dtype=torch.float64),
 'tumor_subtype': ['HER2pos'],
 'her2': tensor([1.], dtype=torch.float64),
 'seg_path': ['/projects/0/prjs1204/data/mamamia/mamamia_processed/pcr/stratified_96_train/segmentations/expert/DUKE_001.nii.gz']}

In [ ]:
import torch
import os

model_name = 'ddp_spade_96_250403'
dir_name = "/projects/0/prjs1204/projects/spadebreast2025/src/spadebreast/checkpoints/"


i = 20
loss =[]
checkpoint = torch.load(
    os.path.join(dir_name, model_name, f"checkpoint_{i}.pth"),
    map_location="cpu",
)
loss+= checkpoint["best_loss"]


0.013957204948164133